In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random

In [25]:
class PINN(tf.keras.Model):
    def __init__(self, layers, regularizer_rate=1e-4):
        super(PINN, self).__init__()
        self.nn_layers = layers  # Cambié "layers" a "nn_layers" para evitar conflictos
        self.regularizer = tf.keras.regularizers.l2(regularizer_rate)
        self.nn_model = self.build_model()

    def build_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(1,)))
        for width in self.nn_layers[1:-1]:  # Iterar hasta la penúltima capa para agregar BatchNorm antes de la última capa
            model.add(tf.keras.layers.Dense(width, activation=tf.nn.tanh,  # Cambiado a función de activación swish
                                            kernel_initializer='glorot_normal', 
                                            kernel_regularizer=self.regularizer))
            model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(self.nn_layers[-1], activation=tf.nn.tanh,  # Activación tanh para la última capa
                                        kernel_initializer='glorot_normal', 
                                        kernel_regularizer=self.regularizer))
        return model

    def call(self, inputs):
        return self.nn_model(inputs)

In [33]:
def loss_fn(model, t_inputs, outputs, epsilon, u, DL, K, t_p, Cin):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t_inputs)
        predicted = model(t_inputs)
        q_bar = K * predicted
        dC_dt = tape.gradient(predicted, t_inputs)
        dC_dz = tape.gradient(predicted, t_inputs)
        dC_dz2 = tape.gradient(dC_dz, t_inputs)

    # Balance of non-stationary mass solute equation
    eq1 = dC_dt + (1 - epsilon) / epsilon * q_bar + u / epsilon * dC_dz - DL * dC_dz2
    loss_eq1 = tf.reduce_mean(tf.square(eq1))
    
    # Boundary conditions: Danckwerts at the entrance and exit of the column
    Cp = tf.where(t_inputs <= t_p, Cin, 0.0)
    boundary_loss_1 = tf.square(predicted[0] - Cp + epsilon * DL / u * dC_dz[0])
    boundary_loss_2 = tf.square(dC_dz[-1])
    
    # Combine the boundary losses
    loss_boundary = boundary_loss_1 + boundary_loss_2

    # Combine losses
    combined_loss = loss_eq1*4 + loss_boundary*0.00001

    return combined_loss

In [34]:
# Definición de hiperparámetros y otros valores conocidos
layers = [1] +[30] * 5 + [1]  # Aumentar la complejidad del modelo
model = PINN(layers)

# Decaimiento exponencial de la tasa de aprendizaje
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1, 
    decay_steps=1000, 
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

epsilon = 0.37  # Porosidad del lecho
u = 5.55  # Velocidad superficial del fluido
t_p = 1.0  # Tiempo del pulso de inyección
Cin = 523.23  # Concentración del pulso de analito inyectada en la muestra

# Inicializar DL y K como variables entrenables
DL = tf.Variable(initial_value=-50.0, trainable=True, dtype=tf.float32)
K = tf.Variable(initial_value=-50.0, trainable=True, dtype=tf.float32)

# Cargar datos de entrenamiento desde el Excel
data = pd.read_excel("datos_cromatografia.xlsx", sheet_name="training, Cin = 523.23 mg L^-1")

# Normalizar los datos de entrenamiento
t_train = tf.convert_to_tensor(data['min'].to_numpy().reshape(-1, 1) / data['min'].max(), dtype=tf.float32)  # tiempo
C_train = tf.convert_to_tensor(data['AU'].to_numpy().reshape(-1, 1) / data['AU'].max(), dtype=tf.float32)   # concentración

# Entrenamiento
epochs = 15000  # Incrementar el número de épocas

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(model, t_train, C_train, epsilon, u, DL, K, t_p, Cin)
    grads = tape.gradient(loss_value, model.trainable_variables + [DL, K])
    optimizer.apply_gradients(zip(grads, model.trainable_variables + [DL, K]))
    
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch}, Loss: {loss_value.numpy()}, DL: {DL.numpy()}, K: {K.numpy()}")


Epoch: 0, Loss: [[3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.4773]
 [3170.47

KeyboardInterrupt: 